#### local run command
`blaze run -c opt learning/brain/research/babelfish/colab:colab_notebook --define=babelfish_task=multimodal`

In [ ]:
import lingvo.compat as tf
import matplotlib.pyplot as plt
import numpy as np
import pprint
import os

from lingvo.core import py_utils
from google3.learning.brain.research.babelfish import tokenizers
from google3.learning.brain.research.babelfish.multimodal.params.experimental import image_text_baselines as it_params

# from google3.pyglib import gfiler

from google3.perftools.accelerators.xprof.api.colab import xprof

tf.disable_eager_execution()

In [ ]:
# mdl = it_params.ImageText2TextLMSmall()
mdl = it_params.Text2TextLMSmall()
p = mdl.Task()

# Note: We use the name as part of var/name scopes, you need to ensure that
# the name here matches for checkpoints to load successfully.

# p.name = 'ImageText2TextLMTask'
p.name = 'Text2TextLM'
p.input = mdl.Train()

In [ ]:
# We are going to use the global graph for this entire colab.
tf.reset_default_graph()

# Instantiate the Task.
task = p.Instantiate()

# Create variables by running FProp.
_ = task.FPropDefaultTheta()

In [ ]:
# Create a new session and initialize all the variables.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
# Setup the checkpoint loading rules for OverrideVarsFromCheckpoints.
loading_rules = [
    (
        "(.*)",  # Regexp match all variables in the ckpt.
        "%s"     # Format string to use the saved var name as is.
    )
]
ignore_rules = []  # No ignore rules, parse all saved vars.
# ckpt_path = '/cns/jn-d/home/ziruiw/brain/rs=6.3/ImageText2TextLM.small.ibz4096.tbz512.PrefixLM.Res50.Trans2.BatchMajor.RelPos.LR5e4.WD1e2/train/ckpt-01000000'
# ckpt_path = '/cns/tp-d/home/runzheyang/brain/rs=6.3/text2textlm.small/train/ckpt-01000000'

ckpts_loading_rules = {
    ckpt_path: (loading_rules, ignore_rules)
}

# Load the saved checkpoint into the session.
py_utils.OverrideVarsFromCheckpoints(
    tf.all_variables(), ckpts_loading_rules)(sess)

In [ ]:
# feeds = {
#     'text': tf.placeholder(tf.string, shape=[1,])
# }
# input_batch = py_utils.NestedMap(encoder_inputs=feeds['text'], decoder_inputs=feeds['text'])

# mdl.T2T_BATCH_SIZE = 1
# mdl.I2T_BATCH_SIZE = 1

mdl.BATCH_SIZE = 1
input_p = mdl.Train()

input_gen = input_p.Instantiate()
input_batch = input_gen.GetPreprocessedInputBatch()

encoder_inputs = input_batch.encoder_inputs
decoder_inputs = input_batch.decoder_inputs
encoder_paddings = input_batch.encoder_paddings
decoder_paddings = input_batch.decoder_paddings

# encoder
sources = py_utils.NestedMap(
    ids=encoder_inputs, paddings=encoder_paddings)
encoder_embeddings = task.encoder.FPropEmbeddings(task.theta.encoder, sources)
encoder_outputs = task.encoder.FPropTransformerLayers(
    task.theta.encoder, encoder_embeddings)

# decoder
targets = py_utils.NestedMap(
    ids=decoder_inputs, paddings=decoder_paddings)
predictions = task.decoder.ComputePredictions(task.theta.decoder, encoder_outputs, targets)

# Notice that we are calling this with task.theta which ensures that we are
# using the same variables which we have just loaded.
fetches = py_utils.NestedMap(
          {"encoder_inputs": encoder_inputs,
           "sources": sources,
           "encoder_embeddings": encoder_embeddings,
           "encoder_outputs": encoder_outputs,
           "decoder_inputs": decoder_inputs,
           "targets": targets,
           "predictions":predictions})

In [ ]:
fetches

In [ ]:
test_output = sess.run(fetches)

In [ ]:
test_output["encoder_embeddings"]["input_embs"]

In [ ]:
test_output["encoder_embeddings"]["input_embs"].shape